In [3]:
from keras.layers import Input, Convolution2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras import backend as K
from keras.optimizers import SGD
import numpy.random as rng
import numpy as np
import dill as pickle
import matplotlib.pyplot as plt

def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
#//TODO: figure out how to initialize layer biases in keras.

input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Convolution2D(64,10,10,activation='relu',input_shape=input_shape,init=W_init))
convnet.add(MaxPooling2D())
convnet.add(Convolution2D(128,7,7,activation='relu',init=W_init))
convnet.add(MaxPooling2D())
convnet.add(Convolution2D(128,4,4,activation='relu',init=W_init))
convnet.add(MaxPooling2D())
convnet.add(Convolution2D(256,4,4,activation='relu',init=W_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid"))
#encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#merge two encoded inputs with the l1 distance between them
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([encoded_l,encoded_r], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(1,activation='sigmoid')(both)
siamese_net = Model(input=[left_input,right_input],output=prediction)
optimizer = SGD(0.001,momentum=0.5,nesterov=True,decay=0.0003)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)
siamese_net.count_params()
for i in convnet.get_layers():
    print(i)

AttributeError: 'Sequential' object has no attribute 'get_layers'

In [2]:
with open( "/home/soren/keras-oneshot/train.pickle","r") as f:
    (X,y) = pickle.load(f)

with open( "/home/soren/keras-oneshot/val.pickle","r") as f:
    (Xval,yval) = pickle.load(f)
print(X.shape)
a=siamese_net.predict([X[1,1,:,:].reshape(1,105,105,1),X[7,1,:,:].reshape(1,105,105,1) ])
b=siamese_net.predict([X[7,1,:,:].reshape(1,105,105,1),X[1,1,:,:].reshape(1,105,105,1) ])


ValueError: too many values to unpack

In [ ]:
-
    


In [ ]:
loader = Siamese_Loader(X,Xval)

def concat_images(X):
    """Concatenates a bnch of images into a big matrix for plotting purposes."""
    nc,h,w,_ = X.shape
    X = X.reshape(nc,h,w)
    n = np.ceil(np.sqrt(nc)).astype("int8")
    img = np.zeros((n*w,n*h))
    x = 0
    y = 0
    for example in range(nc):
        img[x*w:(x+1)*w,y*h:(y+1)*h] = X[example]
        y += 1
        if y >= n:
            y = 0
            x += 1
    return img


def plot_oneshot_task(pairs):
    """Takes a one-shot task given to a siamese net and  """
    fig,(ax1,ax2) = plt.subplots(2)
    ax1.matshow(pairs[0][0].reshape(105,105),cmap='gray')
    img = concat_images(pairs[1])
    ax1.get_yaxis().set_visible(False)
    ax1.get_xaxis().set_visible(False)
    ax2.matshow(img,cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.show()
#example of a one-shot learning task
pairs, targets = loader.make_oneshot_task(25)
plot_oneshot_task(pairs)


In [ ]:
evaluate_every = 250
loss_every=40
batch_size = 128
N_way = 20
n_val = 60
siamese_net.load_weights("/home/soren/keras-oneshot/weights")
for i in range(90000):
    (inputs,targets)=loader.get_batch(batch_size)
    loss=siamese_net.train_on_batch(inputs,targets)
    if i % evaluate_every == 0:
        val_acc = loader.test_oneshot(siamese_net,N_way,n_val,verbose=True)
    if i % loss_every == 0:
        print("iteration {}, training loss: {},".format(i,loss))

        


In [ ]:
siamese_net.save('/home/soren/keras-oneshot/weights')
        
    